In [183]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pyodbc
import pandas_access as pa
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import RandomForestClassifier,HistGradientBoostingClassifier,VotingClassifier,StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score,classification_report
from sklearn.model_selection import cross_val_predict,GridSearchCV, StratifiedKFold
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler  
from sklearn.neural_network import MLPClassifier
from tensorflow import keras
import tensorflow as tf
import imblearn
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN,SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbpipeline
import seaborn as sns
from collections import Counter
import loras
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer,KNNImputer

In [51]:
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)

In [52]:
dataset_close=pd.read_csv('data\hospital_closure\hospital_closure.csv')

In [53]:
MDB_2017='data\hospital_compare\hc_apr2017\Hospital.mdb'
MDB_2016 = 'data\hospital_compare\hc_may2016\Hospital.mdb'
MDB_2015='data\hospital_compare\Copy of HospitalMay2015.mdb'
MDB_2014='data\hospital_compare\HC_May2014.mdb'

DRV = '{Microsoft Access Driver (*.mdb, *.accdb)}'
PWD = 'pw'

con_2017 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2017,PWD))
con_2016 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2016,PWD))
con_2015 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2015,PWD))
con_2014 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2014,PWD))
cur_2017 = con_2017.cursor()
cur_2016 = con_2016.cursor()
cur_2015 = con_2015.cursor()
cur_2014 = con_2014.cursor()

In [54]:
table_name = 'HQI_HOSP_ReadmDeath'
table_name2='HQI_HOSP_ReadmCompDeath'
table_name3='HQI_HOSP_Comp'
query = "SELECT * FROM {}".format(table_name)
query2="SELECT * FROM {}".format(table_name2)
query3="SELECT * FROM {}".format(table_name3)
rows_2017 = cur_2017.execute(query).fetchall()
rows_2017_c=cur_2017.execute(query3).fetchall()
rows_2016 = cur_2016.execute(query).fetchall()
rows_2016_c = cur_2016.execute(query3).fetchall()
rows_2015 = cur_2015.execute(query2).fetchall()
#rows_2014 = cur_2014.execute(query2).fetchall()

In [552]:
arr_2017=np.array(rows_2017)
dataset_2017=pd.DataFrame(arr_2017[:,[0,3,6]],columns=['id','score_name','Score'])
arr_2017_c=np.array(rows_2017_c)
dataset_2017_c=pd.DataFrame(arr_2017_c[:,[0,3,6]],columns=['id','score_name','Score'])
arr_2016=np.array(rows_2016)
dataset_2016=pd.DataFrame(arr_2016[:,[0,3,6]],columns=['id','score_name','Score'])
arr_2016_c=np.array(rows_2016_c)
dataset_2016_c=pd.DataFrame(arr_2016_c[:,[0,3,6]],columns=['id','score_name','Score'])
arr_2015=np.array(rows_2015)
dataset_2015=pd.DataFrame(arr_2015[:,[0,3,6]],columns=['id','score_name','Score'])
#dataset_2014=pd.DataFrame(arr_2014[:,[1,5,6,7,8,9,10]],columns=['id','1','2','3','4','5','6'])

In [553]:
dataset_2015=dataset_2015[dataset_2015['id'].map(lambda x: x.find('F')==-1)]
dataset_2015.drop(dataset_2015[dataset_2015.id=='670092'].index,inplace=True)
dataset_2016=dataset_2016[dataset_2016['id'].map(lambda x: x.find('F')==-1)]
dataset_2016_c=dataset_2016_c[dataset_2016_c['id'].map(lambda x: x.find('F')==-1)]

In [554]:
dataset_2016=pd.concat([dataset_2016,dataset_2016_c],axis=0)
dataset_2017=pd.concat([dataset_2017,dataset_2017_c],axis=0)

In [555]:
dataset_2015.sort_values(by=['id','score_name'],inplace=True)
dataset_2016.sort_values(by=['id','score_name'],inplace=True)
dataset_2017.sort_values(by=['id','score_name'],inplace=True)

In [556]:
da_2015=dataset_2015[['id','Score']].to_numpy().reshape(4654,38)
da_2016=dataset_2016[['id','Score']].to_numpy().reshape(4643,42)
da_2017=dataset_2017[['id','Score']].to_numpy().reshape(4805,50)

In [557]:
arr1=[i*2 for i in range(19)]
arr2=[i*2 for i in range(21)]
arr3=[i*2 for i in range(25)]

In [558]:
da_2015=np.delete(da_2015,arr1[1:],axis=1)
da_2016=np.delete(da_2016,arr2[1:],axis=1)
da_2017=np.delete(da_2017,arr3[1:],axis=1)

In [559]:
da_2015=pd.DataFrame(da_2015,columns=['id','COMP_HIP_KNEE','MORT_30_AMI','MORT_30_COPD','MORT_30_HF','MORT_30_PN','MORT_30_STK','PSI_12_POSTOP_PULMEMB_DVT','PSI_14_POSTOP_DEHIS','PSI_15_ACC_LAC','PSI_4_SURG_COMP','PSI_6_IAT_PTX','PSI_90_SAFETY','READM_30_AMI','READM_30_COPD','READM_30_HF','READM_30_HIP_KNEE','READM_30_HOSP_WIDE','READM_30_PN','READM_30_STK'])
da_2016=pd.DataFrame(da_2016,columns=['id','COMP_HIP_KNEE','MORT_30_AMI','MORT_30_CABG','MORT_30_COPD','MORT_30_HF','MORT_30_PN','MORT_30_STK','PSI_12_POSTOP_PULMEMB_DVT','PSI_14_POSTOP_DEHIS','PSI_15_ACC_LAC','PSI_4_SURG_COMP','PSI_6_IAT_PTX','PSI_90_SAFETY','READM_30_AMI','READM_30_CABG','READM_30_COPD','READM_30_HF','READM_30_HIP_KNEE','READM_30_HOSP_WIDE','READM_30_PN','READM_30_STK'])
da_2017=pd.DataFrame(da_2017,columns=['id','COMP_HIP_KNEE','MORT_30_AMI','MORT_30_CABG','MORT_30_COPD','MORT_30_HF','MORT_30_PN','MORT_30_STK','PSI_12_POSTOP_PULMEMB_DVT','PSI_13_POST_SEPSIS','PSI_14_POSTOP_DEHIS','PSI_15_ACC_LAC','PSI_3_ULCER','PSI_4_SURG_COMP','PSI_6_IAT_PTX','PSI_7_CVCBI','PSI_8_POST_HIP','PSI_90_SAFETY','READM_30_AMI','READM_30_CABG','READM_30_COPD','READM_30_HF','READM_30_HIP_KNEE','READM_30_HOSP_WIDE','READM_30_PN','READM_30_STK'])

In [560]:
da_2016.drop(columns=['MORT_30_CABG','READM_30_CABG'],inplace=True)
da_2017.drop(columns=['MORT_30_CABG','PSI_13_POST_SEPSIS','PSI_3_ULCER','PSI_7_CVCBI','PSI_8_POST_HIP','READM_30_CABG'],inplace=True)


In [561]:
close_2017=dataset_close[dataset_close['year_close_combined']==2017]
close_2016=dataset_close[dataset_close['year_close_combined']==2016]
close_2015=dataset_close[dataset_close['year_close_combined']==2015]
close_2014=dataset_close[dataset_close['year_close_combined']==2014]
close_2013=dataset_close[dataset_close['year_close_combined']==2013]

close_2015_=pd.concat([close_2017,close_2016])
close_2014_=pd.concat([close_2016,close_2015])
close_2013_=pd.concat([close_2015,close_2014])

#close_2015_=pd.concat([close_2017,close_2016,close_2015])
#close_2014_=pd.concat([close_2016,close_2015,close_2014])
#close_2013_=pd.concat([close_2015,close_2014,close_2013])
da_2017['Closure']=da_2017.id.apply(lambda x:1 if int(x) in np.array(close_2015_['id']) else 0)
da_2017.drop(da_2017[da_2017['id']==any(close_2013['id'])].index,inplace=True)
da_2016['Closure']=da_2016.id.apply(lambda x:1 if int(x) in np.array(close_2014_['id']) else 0)
da_2015['Closure']=da_2015.id.apply(lambda x:1 if int(x) in np.array(close_2013_['id']) else 0)


In [66]:
close_2016=dataset_close[dataset_close['year_close_combined']==2016]
close_2015=dataset_close[dataset_close['year_close_combined']==2015]
close_2014=dataset_close[dataset_close['year_close_combined']==2014]
close_=pd.concat([close_2016,close_2015,close_2014])
da_2017['Closure']=da_2017.id.apply(lambda x:1 if int(x) in np.array(close_['id']) else 0)
da_2016['Closure']=da_2016.id.apply(lambda x:1 if int(x) in np.array(close_['id']) else 0)
da_2015['Closure']=da_2015.id.apply(lambda x:1 if int(x) in np.array(close_['id']) else 0)

In [355]:
da_2017[da_2017['Closure']==1]

,id,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
1012,110205,NaN,NaN,7.3,12.1,20.9,NaN,5.82,NaN,1.21,NaN,0.40,0.89,NaN,21.2,23.6,NaN,15.8,18.3,NaN,1
2329,250084,3.5,15.9,8.4,14.2,19.4,15.0,4.67,2.24,1.43,NaN,0.43,0.85,17.8,20.5,22.0,4.8,15.9,17.0,13.5,1
4030,450214,NaN,NaN,8.3,11.2,16.5,NaN,4.72,2.29,1.24,NaN,0.40,0.82,NaN,20.2,20.1,NaN,15.8,15.9,NaN,1
4101,450605,NaN,NaN,8.2,10.7,15.2,NaN,NaN,NaN,1.69,NaN,0.47,0.96,NaN,18.4,21.3,NaN,14.7,15.1,NaN,1
4147,450749,NaN,NaN,8.6,10.9,15.2,NaN,NaN,NaN,1.41,NaN,0.41,0.90,NaN,20.5,22.4,NaN,15.9,17.9,NaN,1


In [356]:
da_2016[da_2016['Closure']==1]

,id,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
356,050196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
439,050435,2.9,NaN,7.9,10.6,12.3,NaN,3.91,1.64,1.70,NaN,0.37,0.74,NaN,19.7,22.0,4.4,14.4,17.2,NaN,1
1092,140026,2.9,13.6,7.8,10.7,12.2,15.2,4.40,1.60,1.06,NaN,0.36,0.64,17.2,20.2,20.8,4.7,15.5,17.6,12.1,1
1492,170010,NaN,NaN,7.8,13.1,12.3,15.4,4.85,NaN,1.85,NaN,0.38,0.84,NaN,19.2,20.5,NaN,15.8,15.1,11.9,1
1675,180117,NaN,NaN,7.3,12.0,13.4,NaN,3.80,NaN,1.58,NaN,0.38,0.72,NaN,19.7,24.2,NaN,15.2,17.8,NaN,1
2265,250084,3.6,15.1,8.5,13.7,16.1,16.0,3.80,1.64,1.65,NaN,0.39,0.76,18.1,20.5,22.2,4.7,14.9,16.9,13.5,1
2368,260116,4.4,NaN,5.8,10.6,13.7,15.1,4.73,1.64,1.79,NaN,0.37,0.81,NaN,21.4,22.7,5.4,15.6,18.0,NaN,1
2582,290020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.80,NaN,0.39,0.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3250,370051,NaN,NaN,6.8,10.2,10.8,NaN,NaN,NaN,1.75,NaN,0.38,0.80,NaN,20.0,20.9,NaN,15.3,17.0,NaN,1
3278,370169,NaN,NaN,NaN,NaN,11.9,NaN,NaN,NaN,1.79,NaN,0.39,0.81,NaN,19.6,NaN,NaN,15.7,18.1,NaN,1


In [357]:
da_2015[da_2015['Closure']==1]

,id,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
205,040042,NaN,14.2,9.4,11.3,14.1,16.2,4.67,1.80,1.81,NaN,0.37,0.80,NaN,22.1,23.1,NaN,17.2,19.3,12.6,1
443,050435,2.8,NaN,8.0,10.4,11.6,NaN,4.11,1.87,2.10,NaN,0.39,0.84,NaN,20.4,22.6,4.3,15.7,17.7,NaN,1
834,100239,3.7,15.0,7.1,12.5,11.0,14.1,4.43,1.86,1.53,103.97,0.42,0.73,NaN,19.6,22.3,5.6,15.4,16.9,14.0,1
977,110183,NaN,NaN,7.0,12.9,12.2,NaN,3.82,1.87,1.47,NaN,0.39,0.70,NaN,21.0,23.2,NaN,15.0,17.3,NaN,1
1095,140026,3.3,14.2,6.7,9.8,12.3,16.8,3.77,1.80,1.11,NaN,0.38,0.62,18.7,22.3,21.0,5.1,15.7,18.9,13.0,1
1497,170010,3.2,NaN,7.9,13.4,15.1,15.6,4.91,1.88,1.94,NaN,0.40,0.88,NaN,19.5,22.1,5.3,14.9,16.8,13.0,1
1679,180117,NaN,NaN,7.5,11.3,14.9,NaN,4.26,NaN,1.72,NaN,0.40,0.79,NaN,21.2,24.1,NaN,16.9,18.5,NaN,1
1689,180149,NaN,NaN,6.3,11.5,11.2,NaN,NaN,NaN,1.76,NaN,0.41,0.83,NaN,22.8,27.4,NaN,16.7,19.2,NaN,1
1725,190009,NaN,NaN,NaN,NaN,NaN,NaN,4.54,NaN,2.19,NaN,0.41,0.92,NaN,NaN,NaN,NaN,16.1,17.9,NaN,1
1843,200025,3.4,14.4,7.3,11.5,10.0,NaN,3.89,1.86,1.82,NaN,0.40,0.79,NaN,20.5,22.6,5.1,15.7,16.5,NaN,1


In [562]:
da_2017=da_2017.replace({'Not Available':np.nan,' Not Available':np.nan})
da_2016=da_2016.replace({'Not Available':np.nan,' Not Available':np.nan})
da_2015=da_2015.replace({'Not Available':np.nan,' Not Available':np.nan})

In [563]:
da_=pd.concat([da_2017,da_2016,da_2015])

In [564]:
da_['id']=pd.to_numeric(da_['id']).astype(int)
da_.loc[da_['id']==40021,'Closure']=0
da_.loc[da_['id']==200025,'Closure']=0
da_.loc[da_['id']==260147,'Closure']=0
da_.loc[da_['id']==440047,'Closure']=0
da_.loc[da_['id']==180149,'Closure']=0
da_.loc[da_['id']==450832,'Closure']=0
da_.loc[da_['id']==670052,'Closure']=0
da_.drop(columns=['id'],inplace=True)

In [565]:
for col in da_:
    da_[col]=pd.to_numeric(da_[col])

In [202]:
da_

,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
0,3.8,12.5,9.3,12.4,15.5,15.4,3.32,2.72,1.35,168.26,0.36,0.68,16.5,21.1,21.4,5.1,15.4,18.7,12.7,0
1,3.0,16.0,7.6,15.5,20.8,15.5,5.63,2.11,1.25,179.05,0.47,0.85,16.7,18.0,21.9,5.7,14.9,16.4,13.4,0
2,3.8,16.7,7.1,15.6,18.2,17.9,3.85,2.50,1.93,198.33,0.41,0.91,16.1,19.8,20.6,5.0,15.4,17.9,12.0,0
3,NaN,NaN,9.3,14.4,18.8,16.6,4.57,NaN,1.20,NaN,0.40,0.79,NaN,19.9,21.1,NaN,16.6,17.3,12.7,0
4,NaN,NaN,8.2,12.7,15.7,NaN,NaN,NaN,1.41,NaN,0.41,0.90,NaN,19.2,23.1,NaN,15.7,16.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4649,NaN,NaN,NaN,NaN,NaN,NaN,4.44,NaN,1.76,NaN,0.40,0.92,NaN,NaN,NaN,NaN,14.9,NaN,NaN,0
4650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [76]:
list(da_.columns[:-1])

['COMP_HIP_KNEE',
 'MORT_30_AMI',
 'MORT_30_COPD',
 'MORT_30_HF',
 'MORT_30_PN',
 'MORT_30_STK',
 'PSI_12_POSTOP_PULMEMB_DVT',
 'PSI_14_POSTOP_DEHIS',
 'PSI_15_ACC_LAC',
 'PSI_4_SURG_COMP',
 'PSI_6_IAT_PTX',
 'PSI_90_SAFETY',
 'READM_30_AMI',
 'READM_30_COPD',
 'READM_30_HF',
 'READM_30_HIP_KNEE',
 'READM_30_HOSP_WIDE',
 'READM_30_PN',
 'READM_30_STK']

In [362]:
da_.describe()

,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
count,8233.000000,7282.000000,10900.000000,10995.000000,12275.000000,8284.000000,9036.000000,7751.000000,9631.000000,5502.000000,9650.000000,9706.000000,6632.000000,11103.000000,11212.000000,8266.000000,13305.000000,12309.000000,8109.000000,14102.000000
mean,3.185461,14.363142,7.913459,11.959345,13.345222,15.044532,4.518752,1.978051,1.726129,124.304022,0.400696,0.848702,17.252141,20.345375,22.234267,4.895717,15.470943,17.133959,12.886496,0.003120
std,0.609817,1.343128,1.067743,1.469456,2.866427,1.712835,1.722131,0.435237,0.644749,20.741338,0.072136,0.183101,1.119150,1.313005,1.610627,0.664587,0.889226,1.292103,1.174144,0.055773
min,1.400000,9.400000,4.600000,6.000000,6.400000,8.600000,0.860000,0.860000,0.260000,56.260000,0.180000,0.370000,13.100000,15.500000,15.800000,2.400000,10.800000,12.900000,8.700000,0.000000
25%,2.800000,13.500000,7.200000,11.000000,11.100000,13.900000,3.430000,1.650000,1.300000,109.320000,0.360000,0.730000,16.500000,19.500000,21.200000,4.500000,14.900000,16.300000,12.100000,0.000000
50%,3.100000,14.300000,7.800000,11.900000,12.800000,14.900000,4.200000,1.880000,1.630000,123.605000,0.390000,0.830000,17.200000,20.300000,22.100000,4.800000,15.400000,17.000000,12.800000,0.000000
75%,3.500000,15.200000,8.500000,12.900000,15.400000,16.100000,5.200000,2.230000,2.020000,137.985000,0.430000,0.930000,18.000000,21.100000,23.200000,5.300000,15.900000,17.900000,13.500000,0.000000
max,6.900000,20.600000,14.100000,18.500000,26.800000,23.800000,20.880000,4.980000,7.220000,212.160000,0.880000,2.140000,21.700000,28.000000,31.700000,9.400000,21.400000,24.700000,18.500000,1.000000


In [363]:
da_[da_['Closure']==0].describe()

,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
count,8221.000000,7273.000000,10863.000000,10957.000000,12235.000000,8271.000000,9010.000000,7733.000000,9588.000000,5500.000000,9607.000000,9662.000000,6625.000000,11063.000000,11174.000000,8254.000000,13265.000000,12268.000000,8097.000000,14058.0
mean,3.185099,14.362711,7.914269,11.959140,13.346596,15.043659,4.519696,1.978374,1.726448,124.310815,0.400687,0.848923,17.251366,20.345774,22.233775,4.895323,15.470863,17.133306,12.886069,0.0
std,0.609950,1.343669,1.068279,1.469459,2.867720,1.713782,1.724203,0.435590,0.645932,20.742026,0.072229,0.183411,1.119373,1.314243,1.611879,0.664577,0.889822,1.292903,1.174580,0.0
min,1.400000,9.400000,4.600000,6.000000,6.400000,8.600000,0.860000,0.860000,0.260000,56.260000,0.180000,0.370000,13.100000,15.500000,15.800000,2.400000,10.800000,12.900000,8.700000,0.0
25%,2.800000,13.500000,7.200000,11.000000,11.100000,13.900000,3.430000,1.650000,1.300000,109.327500,0.360000,0.730000,16.500000,19.500000,21.200000,4.500000,14.900000,16.300000,12.100000,0.0
50%,3.100000,14.300000,7.800000,11.900000,12.800000,14.900000,4.200000,1.880000,1.630000,123.610000,0.390000,0.830000,17.200000,20.300000,22.100000,4.800000,15.400000,17.000000,12.800000,0.0
75%,3.500000,15.200000,8.500000,12.900000,15.400000,16.100000,5.200000,2.230000,2.020000,137.990000,0.430000,0.930000,18.000000,21.100000,23.200000,5.300000,15.900000,17.900000,13.500000,0.0
max,6.900000,20.600000,14.100000,18.500000,26.800000,23.800000,20.880000,4.980000,7.220000,212.160000,0.880000,2.140000,21.700000,28.000000,31.700000,9.400000,21.400000,24.700000,18.500000,0.0


In [402]:
da_[da_['Closure']==1].describe()

,COMP_HIP_KNEE,MORT_30_AMI,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_12_POSTOP_PULMEMB_DVT,PSI_14_POSTOP_DEHIS,PSI_15_ACC_LAC,PSI_4_SURG_COMP,PSI_6_IAT_PTX,PSI_90_SAFETY,READM_30_AMI,READM_30_COPD,READM_30_HF,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,READM_30_PN,READM_30_STK,Closure
count,12.000000,9.000000,37.000000,38.000000,40.000000,13.000000,26.000000,18.000000,43.000000,2.000000,43.000000,44.000000,7.000000,40.000000,38.000000,12.000000,40.000000,41.000000,12.000000,44.0
mean,3.433333,14.711111,7.675676,12.018421,12.925000,15.600000,4.191538,1.838889,1.655116,105.625000,0.402791,0.800227,17.985714,20.235000,22.378947,5.166667,15.497500,17.329268,13.175000,1.0
std,0.465800,0.755719,0.876358,1.487217,2.433079,0.779957,0.632757,0.200261,0.267887,2.340523,0.047275,0.079606,0.530498,0.913025,1.193486,0.641494,0.670433,1.018637,0.812544,0.0
min,2.800000,13.600000,5.800000,9.800000,9.600000,14.100000,2.570000,1.600000,1.060000,103.970000,0.350000,0.620000,17.200000,18.400000,20.000000,4.300000,14.000000,15.100000,11.900000,1.0
25%,3.125000,14.200000,7.100000,11.000000,11.275000,15.100000,3.805000,1.662500,1.470000,104.797500,0.380000,0.747500,17.650000,19.600000,21.700000,4.700000,15.000000,16.900000,12.525000,1.0
50%,3.400000,14.900000,7.800000,11.550000,12.250000,15.400000,4.155000,1.835000,1.690000,105.625000,0.390000,0.810000,18.100000,20.200000,22.250000,5.150000,15.600000,17.300000,13.250000,1.0
75%,3.700000,15.100000,8.200000,12.975000,14.275000,16.100000,4.637500,1.870000,1.820000,106.452500,0.410000,0.860000,18.300000,20.850000,23.175000,5.450000,15.825000,17.900000,13.925000,1.0
max,4.400000,15.900000,9.800000,15.700000,20.900000,16.900000,5.820000,2.290000,2.190000,107.280000,0.660000,0.960000,18.700000,22.300000,24.400000,6.300000,17.200000,19.700000,14.200000,1.0


In [453]:
da_.drop(columns=['PSI_4_SURG_COMP','READM_30_AMI'],inplace=True)
#'COMP_HIP_KNEE','MORT_30_AMI','MORT_30_STK''READM_30_HIP_KNEE'

In [505]:
imputer = KNNImputer()
dataset_=imputer.fit_transform(da_)

In [566]:
imputer_i=IterativeImputer(max_iter=30)
imputer_i.fit(da_)
da_=imputer_i.transform(da_)
dataset_=da_

In [486]:
da_.fillna(da_.mean(),inplace=True)
dataset_=da_.to_numpy()

In [567]:
train, test = train_test_split(dataset_, test_size=0.3)
X_train = train[:,:-1]
Y_train = train[:,-1].astype(int)
X_test  = test[:,:-1]
Y_test  = test[:,-1].astype(int)
X_train.shape, Y_train.shape,X_test.shape, Y_test.shape

((9871, 19), (9871,), (4231, 19), (4231,))

In [568]:
smote = SMOTE()
x_smote , y_smote = smote.fit_sample(X_train, Y_train)
x_train1, y_train1 = x_smote , y_smote
x_test1 =X_test

In [569]:
m=XGBClassifier()
m.fit(x_train1, y_train1)
predict_y = m.predict(x_test1)
print('ROCAUC score:',roc_auc_score(Y_test, predict_y))
print('Accuracy score:',accuracy_score(Y_test, predict_y))
print('F1 score:',f1_score(Y_test, predict_y))
print(classification_report(Y_test, predict_y))
confusion_matrix(Y_test, predict_y)

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:50:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROCAUC score: 0.8175894011202067
Accuracy score: 0.9978728432994564
F1 score: 0.6086956521739131
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4220
           1       0.58      0.64      0.61        11

    accuracy                           1.00      4231
   macro avg       0.79      0.82      0.80      4231
weighted avg       1.00      1.00      1.00      4231



array([[4215,    5],
       [   4,    7]], dtype=int64)

In [570]:
X_train, X_test, y_train, y_test = train_test_split(dataset_[:,:-1],
                                                    dataset_[:,-1],
                                                    test_size=0.05,
                                                    stratify=dataset_[:,-1]
                                                    )
stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )

In [529]:
pipeline = imbpipeline(steps = [['smote', SMOTE()],
                                
                                ['classifier', RandomForestClassifier()]])
    
param_grid = {'classifier__n_estimators':[10,20,30,40,50,60,70,80,90,100,150,200,300,400,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_s = grid_search.best_score_
test_score_RF_s = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_s}\nTest score: {test_score_RF_s}')

Cross-validation score: 0.4413986013986014
Test score: 1.0


In [530]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )
    
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_a = grid_search.best_score_
test_score_RF_a = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_a}\nTest score: {test_score_RF_a}')

Cross-validation score: 0.41216783216783215
Test score: 1.0


In [533]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )
    
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_svm = grid_search.best_score_
test_score_RF_svm = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_svm}\nTest score: {test_score_RF_svm}')

Cross-validation score: 0.27939393939393936
Test score: 0.6666666666666666


In [534]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_s = grid_search.best_score_
test_score_LR_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_s}\nTest score: {test_score_LR_s}')

Cross-validation score: 0.030404519651552598
Test score: 0.03076923076923077


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [535]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_a = grid_search.best_score_
test_score_LR_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_a}\nTest score: {test_score_LR_a}')

Cross-validation score: 0.029091498941816618
Test score: 0.031007751937984496


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [536]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_svm = grid_search.best_score_
test_score_LR_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_svm}\nTest score: {test_score_LR_svm}')

Cross-validation score: 0.018257794855419663
Test score: 0.029629629629629624


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [543]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[1,2,3,4,5,10,20,30,50,100,200,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_s = grid_search.best_score_
test_score_DT_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_s}\nTest score: {test_score_DT_s}')

Cross-validation score: 0.3005050505050505
Test score: 0.3333333333333333


In [541]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[1,2,3,4,5,10,20,30,50,100,200,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_a = grid_search.best_score_
test_score_DT_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_a}\nTest score: {test_score_DT_a}')

Cross-validation score: 0.331133382746286
Test score: 0.5714285714285715


In [540]:
pipeline = imbpipeline(steps = [['SVMSMOTE',SVMSMOTE()],
                                
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[1,2,3,4,5,10,20,30,50,100,200,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_svm = grid_search.best_score_
test_score_DT_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_svm}\nTest score: {test_score_DT_svm}')

Cross-validation score: 0.2715294117647059
Test score: 0.28571428571428575


In [572]:
pipeline = imbpipeline(steps = [['SMOTE',SMOTE()],
                                
                                ['classifier', XGBClassifier()]])
    
param_grid = {'classifier__min_child_weight': [1, 5, 10]
        }
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_s = grid_search.best_score_
test_score_xgb_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_s}\nTest score: {test_score_xgb_s}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:51:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.6029284441049148
Test score: 0.8


In [574]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', XGBClassifier()]])

grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_a = grid_search.best_score_
test_score_xgb_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_a}\nTest score: {test_score_xgb_a}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:52:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.5963725490196079
Test score: 0.5714285714285715


In [576]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                
                                ['classifier', XGBClassifier()]])
param_grid = {'classifier__min_child_weight': [1, 5, 10]
        }    
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_svm = grid_search.best_score_
test_score_xgb_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_svm}\nTest score: {test_score_xgb_svm}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:53:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.5013986013986014
Test score: 0.4


In [549]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', HistGradientBoostingClassifier()]])
    
param_grid = {'classifier__min_samples_leaf': [1,2,3,4, 5, 10]
        }
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_s = grid_search.best_score_
test_score_xgb_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_s}\nTest score: {test_score_xgb_s}')

Cross-validation score: 0.5049127343244991
Test score: 0.8


In [184]:
stacking_clf=StackingClassifier(estimators=[('rf',RandomForestClassifier()),('xg',XGBClassifier()),('dt',DecisionTreeClassifier()),('hg',HistGradientBoostingClassifier())])
pipeline = imbpipeline(steps = [['SMOTE', ADASYN()],
                                
                                ['classifier', stacking_clf]])
    
param_grid = {'classifier__rf__n_estimators':[10,20,30,40,50,60,70,80,90,100,150,200,300,400,500],'classifier__xg__min_child_weight': [1, 5, 10],'classifier__hg__min_samples_leaf': [1, 5, 10]
        }
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_s = grid_search.best_score_
test_score_xgb_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_s}\nTest score: {test_score_xgb_s}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:34:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:35:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[01:35:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:35:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:35:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.5655011655011655
Test score: 0.5384615384615384


In [95]:

#voting.get_params().keys()

dict_keys(['estimators', 'flatten_transform', 'n_jobs', 'verbose', 'voting', 'weights', 'rf', 'xg', 'dt', 'hg', 'rf__bootstrap', 'rf__ccp_alpha', 'rf__class_weight', 'rf__criterion', 'rf__max_depth', 'rf__max_features', 'rf__max_leaf_nodes', 'rf__max_samples', 'rf__min_impurity_decrease', 'rf__min_impurity_split', 'rf__min_samples_leaf', 'rf__min_samples_split', 'rf__min_weight_fraction_leaf', 'rf__n_estimators', 'rf__n_jobs', 'rf__oob_score', 'rf__random_state', 'rf__verbose', 'rf__warm_start', 'xg__objective', 'xg__use_label_encoder', 'xg__base_score', 'xg__booster', 'xg__colsample_bylevel', 'xg__colsample_bynode', 'xg__colsample_bytree', 'xg__gamma', 'xg__gpu_id', 'xg__importance_type', 'xg__interaction_constraints', 'xg__learning_rate', 'xg__max_delta_step', 'xg__max_depth', 'xg__min_child_weight', 'xg__missing', 'xg__monotone_constraints', 'xg__n_estimators', 'xg__n_jobs', 'xg__num_parallel_tree', 'xg__random_state', 'xg__reg_alpha', 'xg__reg_lambda', 'xg__scale_pos_weight', 'xg__

In [580]:
models = pd.DataFrame({
    'Model': ['Random Forest', 'Logistic Regression', 
              'Decision Tree',  
              'Gradient Boosting','Stacking'],
    'SMOTE': [cv_score_RF_s,cv_score_LR_s,cv_score_DT_s,cv_score_xgb_s,np.NaN],
    'ADASYN': [cv_score_RF_a,cv_score_LR_a,cv_score_DT_a,cv_score_xgb_a,0.56550116],
    'SVMSMOTE':[cv_score_RF_svm,cv_score_LR_svm,cv_score_DT_svm,cv_score_xgb_svm,np.nan]
    })

In [581]:
models

,Model,SMOTE,ADASYN,SVMSMOTE
0,Random Forest,0.441399,0.412168,0.279394
1,Logistic Regression,0.030405,0.029091,0.018258
2,Decision Tree,0.300505,0.331133,0.271529
3,Gradient Boosting,0.602928,0.596373,0.501399
4,Stacking,NaN,0.565501,NaN
